# Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#Downloading the stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Checking the list of stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
#Loading the dataset to a pandas dataframe
df=pd.read_csv('/Data/fake_news.csv')

In [ ]:
#loading the first 5 rows of the dataframe
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#checking the rows and columns of the dataframe
df.shape

(20800, 5)

There are 20800 rows and 5 columns

In [ ]:
#checking the missing values in the dataset
df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


There are 558 missing values in 'title' and 1957 missing values in 'author' and 39 missing values in 'text'. Since this is a large dataset, we can drop the missing values or fill the missing values with empty string

In [ ]:
#filling the missing values with null string
df.fillna('',inplace=True)

In [ ]:
#checking if missing values are filled
df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [ ]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
#merging the title and author data
df['content']=df['author']+''+df['title']

In [ ]:
print(df['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


In [ ]:
#seperating the data & label columns into X and y
X=df.drop(columns='label',axis=1)
y=df['label']

In [ ]:
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# Stemming:

Stemming is the process of reducing the word to its root word

example:
actor,actress,acting --> act

In [ ]:
#creating a variable port_stem to call the function PorterStemmer()
port_stem = PorterStemmer()

In [ ]:
#Creating a user function called stemming()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

Here we are created the function stemming() with few conditions.

1. re.sub to replace and remove every character except a-z and A-Z with a space
2. stemmed_content.lower() to change all the character into lowercase.
3. stemmed_content.split() to convert all string values into list
4. [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] this will remove all stopwords from the content.
5. ' '.join(stemmed_content) this will join all the list of strings into a single string value with space between each.

In [ ]:
#applying the stemming function to content
df['content']=df['content'].apply(stemming)

In [ ]:
print(df['content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data and the label to X and y column
X=df['content'].values
y=df['label'].values

Initially we created X and y with label column in y and the rest of the columns in X. However, since we only need the 'title' and 'author' features in our X, we are assigning the 'content' feature in X and again assigning 'label' feature in y

In [ ]:
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [ ]:
print(y)

[1 0 1 ... 0 1 1]


In [ ]:
#converting text data into numerical data to feed the value into our ML model
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

Tfidfvectorizer() is function we use to convert text data into numerical data

Tf = Term frequency

idf = inverse document frequency

Tf checks how many times a particular word is repeated in a text document, here 'content' and assigns a value to the word.

idf finds the words that are repeated multiple times that are not significant and reduces its value.

Eg: A review about breed of dogs called as 'Rottweiler'. The review will repeat the word Rottweiler multiple times. However, it doesn't have significance according to the repetition.

In [ ]:
#checking the converted data
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 198373 stored elements and shape (20800, 28419)>
  Coords	Values
  (0, 578)	0.2694167078545384
  (0, 4211)	0.36253203231506576
  (0, 5006)	0.24725958235728157
  (0, 5969)	0.35488202138141456
  (0, 6273)	0.2839932825877812
  (0, 8022)	0.2313366174248873
  (0, 12782)	0.24619727512767192
  (0, 14555)	0.2917725968420029
  (0, 15019)	0.4300622675963931
  (0, 22724)	0.25523360180691607
  (0, 26340)	0.2808837940159642
  (1, 2622)	0.3562953366945267
  (1, 3281)	0.18652439327549428
  (1, 3859)	0.45980466668763476
  (1, 4767)	0.23338756776626793
  (1, 5916)	0.31810058109638056
  (1, 8772)	0.5258635625386451
  (1, 11313)	0.24166773097712638
  (1, 27923)	0.36911845953845024
  (2, 5121)	0.5511414848555652
  (2, 5240)	0.40440534260277944
  (2, 8567)	0.3411947414020896
  (2, 9454)	0.30743020569262086
  (2, 16361)	0.43295215406038445
  (2, 26235)	0.3665032495181434
  :	:
  (20797, 1249)	0.3072223353708335
  (20797, 2257)	0.3357782642976524


# Splitting the data into train and test

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=2, stratify=y)

Here we are splitting the train and test data.

test_size=0.2 means we are allocating 20% of data for test purpose

random_state=2 means how the data is shuffled before splitting

stratify=y it helps you maintain the same class proportions in both the training and test sets

# Training the model

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

LogisticRegression()

Evaluating the model

In [ ]:
#accuracy score on training data
X_train_prediction = lr.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [ ]:
print('Accuracy score of the training data:', training_data_accuracy)

Accuracy score of the training data: 0.971875


In [ ]:
#accuracy score on test data
X_test_prediction = lr.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [ ]:
print('Accuracy score of the test data:', test_data_accuracy)

Accuracy score of the test data: 0.9550480769230769


# Making a predictive system

In [ ]:
X_new = X_test[3]

prediction = lr.predict(X_new)
print(prediction)

if prediction[0]==0:
  print('The news is real')
else:
  print('The news is fake')

[0]
The news is real


Here we are making a predictive model to check whether the news is fake or not.

X_test[ ]  here we give the index value inside the [ ] and it checks and returns the news is fake or real.

In [ ]:
#checking what does index 0 contain
print(y_test[0])

1


Fake news

In [ ]:
#checking what does index 3 contain
print(y_test[3])

0


Real news